# Stitching multiple images

This notebook is used to find a solution to the multi-graph syncronization problem and stitch images together.

In [1]:
import cv2 as cv
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import ipynb.fs.defs.Utils as Utils

In [2]:
def simple_graph_stitching(dataset_name,
                            imgs,
                            T_norm,
                            M, 
                            idx_ref = 0,
                            idxs = None,
                            verbose = True,
                            save_output = True,
                            beautify = True,
                            stitching_dir = "stitched",
                            graph_stitching_dir = "simple_graph_stitching",
                            warp_shape = [10000,10000] ):
    
    output_dir = os.path.join(os.path.join(stitching_dir,dataset_name),graph_stitching_dir)
    if save_output:
        if os.path.isdir(output_dir):   
            shutil.rmtree(output_dir)
        if not os.path.isdir(output_dir):   
            os.makedirs(output_dir)
            
    #u: left eigenvectors
    #d: eigenvalues
    #vh: right eigenvectors
    u,d,vh = np.linalg.svd(M) #Application of SVD to the M matrix
    v = vh.transpose() #Transpose matrix of right eigenvectors
    # Find the three right singular vectors associated
    # with the 3 smallest singular value (so the last 3 columns)
    u_hat = v[:,[-1,-2,-3]]
    
    U = Utils.split_states(u_hat) #Get the state of each node
    
    stitcher = Utils.ImageStitcher(imgs)
    
    H = stitcher.get_homographies_from_states(U, idx_ref)

    _,stitch = stitcher.stitch_images(H, idx_ref, T_norm, idxs , beautify=beautify, size = warp_shape) #Stitch all the images
    
    if save_output:
        cv.imwrite(os.path.join(output_dir,"stitched.jpg"), cv.cvtColor(stitch,cv.COLOR_RGB2BGR))
        
    figure(figsize=(40, 40), dpi=80)
    plt.imshow(stitch,),plt.show() 
    
    return H, stitch